In [3]:
import pandas as pd
data=pd.read_csv("FAOLEX_ALL.csv")
# print(data.keys())
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196437 entries, 0 to 196436
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Record Id                196437 non-null  object
 1   Record URL               196437 non-null  object
 2   Document URL             196424 non-null  object
 3   Title                    196437 non-null  object
 4   Original title           63159 non-null   object
 5   Date of original text    196392 non-null  object
 6   Last amended date        29052 non-null   object
 7   Available website        149557 non-null  object
 8   Language of document     196437 non-null  object
 9   Country/Territory        195503 non-null  object
 10  Regional organizations   1170 non-null    object
 11  Territorial subdivision  31989 non-null   object
 12  Type of text             196437 non-null  object
 13  Repealed                 23182 non-null   object
 14  Abstract            